In [ ]:
import numpy as np
import pandas as pd
m=pd.read_csv("tmdb_5000_movies.csv")
c=pd.read_csv("tmdb_5000_credits.csv")

: 

In [ ]:
c.head()


: 

In [ ]:
m.head()

: 

In [ ]:
c=c.rename(index=str,columns={"movie_id":"id"})
m=m.merge(c,on="id")
m=m.rename(index=str,columns={"title_y":"title"})
m=m[["id","title","overview","genres","keywords","cast","crew"]]


: 

In [ ]:
m=m[['id','title','overview','genres','keywords','cast']]

: 

In [ ]:
m.head()

: 

In [ ]:
m.isnull().sum()

: 

In [ ]:
m.dropna(inplace=True)

: 

In [ ]:
m.isnull().sum()

: 

In [ ]:
m.duplicated().sum()

: 

In [ ]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

: 

In [ ]:
m['genres']=m['genres'].apply(convert)

: 

In [ ]:
m['keywords']=m['keywords'].apply(convert)

: 

In [ ]:
def convert_cast(obj):
    L=[]
    counter=0;
    for i in ast.literal_eval(obj):
        if (counter!=3):
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

: 

In [ ]:
m['cast']=m['cast'].apply(convert_cast)

: 

In [ ]:
m['overview']=m['overview'].apply(lambda x:x.split())

: 

In [ ]:
m['genres']=m['genres'].apply(lambda x:[i.replace(" ","")for i in x])
m['keywords']=m['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
m['cast']=m['cast'].apply(lambda x:[i.replace(" ","")for i in x])

: 

In [ ]:
m['tags']=m['overview']+m['genres']+m['keywords']+m['cast']

: 

In [ ]:
nd=m[['id','title','tags']]

: 

In [ ]:
nd.head()

: 

In [ ]:
nd['tags']=nd['tags'].apply(lambda x:" ".join(x))

: 

In [ ]:
nd['tags']=nd['tags'].apply(lambda x:x.lower())

: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer as CV
cv=CV(max_features=1000,stop_words='english')
vectors=cv.fit_transform(nd['tags']).toarray()

: 

In [ ]:
import nltk
from nltk.stem.porter import  PorterStemmer as PS
ps=PS()
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

: 

In [ ]:
nd['tags']=nd['tags'].apply(stem)

: 

In [ ]:
vectors=cv.fit_transform(nd['tags']).toarray()
from sklearn.metrics.pairwise import cosine_similarity  as CS
s=CS(vectors)


: 

In [ ]:
def recommend(movie):
    movie_index=int(nd[nd['title']==movie].index[0])
    distances=s[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:6]
    for i in movies_list:
        print(nd.iloc[i[0]].title)

: 

In [ ]:
recommend('Batman Begins')

: 

In [ ]:
import pickle

: 

In [ ]:
pickle.dump(nd.to_dict(),open('movies_dict.pkl','wb'))

: 

In [ ]:
pickle.dump(s,open('similarity.pkl','wb'))

: 

: 